In [2]:
from __future__ import print_function, absolute_import, division
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf
from keras.utils import conv_utils
from keras.layers import Input, MaxPooling1D ,Activation,Dense, activations, initializers, Flatten, regularizers, constraints, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply
from keras.utils import to_categorical

In [3]:
class Attention(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.

    Linear learnable weight vector , does dot multiplication on a vector
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).'''

    def __init__(self, weights=None, axis=-1,init='he_normal',**kwargs):
        self.axis = axis
        self.init = initializers.get(init)
        self.kernel = weights
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        if len(input_shape)>2:
            raise ValueError("Input to attention layer hasn't been flattened")
        self.input_dim = input_shape[-1]            
        self.kernel = self.add_weight(shape=(self.input_dim,),
                                      initializer=initializers.Ones(),
                                      name='kernel',
                                      constraint=constraints.NonNeg()
                                      #constraint=constraints.min_max_norm(min_value=0.0, max_value=1.0)
                                      #constraint=constraints.UnitNorm(axis=self.axis)
                                     )
        self.input_spec = InputSpec(min_ndim=2, axes={-1: self.input_dim})            
        self.built = True
    def call(self, inputs):
        output = tf.multiply(inputs,self.kernel)
        return output
    def compute_output_shape(self, input_shape):
        return input_shape
    def get_config(self):
        config = {"kernel": self.kernel, "axis": self.axis}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [45]:
att = Attention(weights=[np.ones(10)],name = 'wow',trainable=True)

In [46]:
att.trainable

True

In [115]:
a = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32))
b = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32).reshape(1,3))
c = tf.convert_to_tensor(np.array([[1,2,3],[4,5,6]]).astype(np.float32).reshape(2,3))
a,b,c

(<tf.Tensor 'Const:0' shape=(3,) dtype=float32>,
 <tf.Tensor 'Const_1:0' shape=(1, 3) dtype=float32>,
 <tf.Tensor 'Const_2:0' shape=(2, 3) dtype=float32>)

In [8]:
out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([a, b])

In [9]:
np.array(out)

array(<tf.Tensor 'lambda_1/mul:0' shape=(1, 3, 3) dtype=float32>,
      dtype=object)

In [10]:
out = tf.multiply(c,a)

In [13]:
out = Multiply()([b,b])

In [14]:
print((tf.Session().run(out)))

[[1. 4. 9.]]


In [49]:
att.get_weights()

[array([-0.1099719,  0.4618892,  0.4047528], dtype=float32)]

In [46]:
-.09050519+.05788201-.47705907

-0.50968225

In [50]:
att.get_config()

{'name': 'attention_6',
 'trainable': True,
 'kernel': <tf.Variable 'attention_6/kernel:0' shape=(3,) dtype=float32_ref>,
 'axis': -1}

In [12]:
def Network():
    input = Input(shape=(10, 1))
    t = BatchNormalization(epsilon=1.1e-5, momentum=.99, axis=-1)(input)
    t = Activation('relu')(t)
    t = Dropout(rate=.5, seed=1)(t)
    t = MaxPooling1D(pool_size=1)(t)
    
    t = Flatten()(t)
    tt = t
    t = Attention(weights = [np.ones(10)],name='att',trainable = True)(t)
    t2 = Attention(weights = [np.ones(10)],name='att2')(tt)
    
    t = Dense(2,activation='softmax',name='out')(t)
    t2 = Dense(2,activation='softmax',name='out2')(t2)
    opt = SGD(lr=.001,decay=.001)
    model = Model(inputs=input, outputs=[t,t2])
    model.compile(optimizer=opt, loss=['categorical_crossentropy','categorical_crossentropy'], metrics=['accuracy'])
    return model

In [13]:
model = Network()

In [14]:
model.summary()
#plot_model(model, to_file='model.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 1)        0                                            
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 10, 1)        4           input_3[0][0]                    
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 10, 1)        0           batch_normalization_3[0][0]      
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 10, 1)        0           activation_3[0][0]               
__________________________________________________________________________________________________
max_poolin

In [15]:
model.get_layer("att").get_weights(),model.get_layer("att2").get_weights()

InternalError: GPU sync failed

In [16]:
x = np.random.rand(100,10,1)
y = np.random.randint(low = 0,high = 2,size = (100,1))
y = to_categorical(y,num_classes=2)

In [17]:
model.fit(x,[y,y],batch_size=10,epochs=5,verbose=1)

Epoch 1/5


InternalError: GPU sync failed

In [53]:
model.get_layer("att" ).get_weights(),model.get_layer("att2" ).get_weights()

([array([1.0006565 , 0.9950268 , 1.0006152 , 0.9775875 , 0.99507576,
         1.0147973 , 0.9942441 , 0.9988031 , 0.96180874, 0.9970449 ],
        dtype=float32)],
 [array([0.9601198 , 0.99470335, 0.99289495, 0.9861508 , 0.9998297 ,
         0.9958785 , 0.96344477, 0.99211186, 0.9646526 , 0.9982504 ],
        dtype=float32)])

In [45]:
model.get_layer("att").trainable= False

In [129]:
a = tf.convert_to_tensor(np.random.rand(5,10).astype(np.float32))

In [130]:
att(a)

<tf.Tensor 'wow_19/Mul:0' shape=(5, 10) dtype=float32>

In [137]:
att.set_weights(model.get_layer("att" ).get_weights())

In [39]:
model.get_layer("att" ).set_kernel(model.get_layer("att2" ).kernel)

Setting kernel


In [42]:
a = model.get_layer("att2" ).kernel

In [1]:
def Confused_Crossentropy(y_true, y_pred):
    y_predfused = tf.multiply(y_pred,0)+.5
    #y_predfused = tf.convert_to_tensor((np.ones((batch,num_class),dtype=np.float32)*0.5))
    #y_truefused = tf.convert_to_tensor( to_categorical(np.ones(batch),num_class) )
    return K.abs(K.categorical_crossentropy(y_true, y_pred)-K.categorical_crossentropy(y_true,y_predfused))

In [68]:
to_categorical(0,num_classes=2)

array([1., 0.], dtype=float32)